# Extract sources from catalogs  in  OGA

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/18
- update : 2024/03/05 : Add very important ``sourceTable_visit``


         DatasetType('isolated_star_cat', {instrument, skymap, tract}, DataFrame),
         DatasetType('isolated_star_sources', {instrument, skymap, tract}, DataFrame),
         DatasetType('isolated_star_sources_atlas_refcat2_20220201_match', {skymap, tract}, DataFrame),


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! eups list -s lsst_distrib

In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.table import Table
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [ ]:
DATE = 202402
#filesource_out="finalized_src_table_202402.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [ ]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
#file_out="sourceTable_202312.csv"

## Give the collection

In [ ]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

## Config

# Sources

In [ ]:
# src : a Source catalog
# srcMatchFull
# finalized_src_table : sape of a pandas dataframe

#'isolated_star_cat', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources_atlas_refcat2_20220201_match', {skymap, tract}, DataFrame),

#data_product = "isolated_star_sources" 
#data_product = "isolated_star_cat" 
#data_product = 'isolated_star_sources_atlas_refcat2_20220201_match' # Nothing
#data_product = "icSrc"
data_product = "sourceTable_visit"
#data_product = "sourceTable_visit_gaia_dr2_2023_07_07_match". Not found
#data_product = 'sourceTable_visit_atlas_refcat2_20220201_match' Not found
#data_product = "src"
datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=my_collection, where= "instrument='LATISS'")
filesource_out=f"{data_product}_{DATE}.csv"

In [ ]:
FLAG_GET_SCHEMA = False

In [ ]:
FLAG_RESTRICT_VISITS = False

if FLAG_RESTRICT_VISITS:
    VISITDATEMIN = 20231121
    VISITDATEMAX = 20231222	
    filesource_out=f"{data_product}_{DATE}_{VISITDATEMIN}-{VISITDATEMAX}.csv"

In [ ]:
all_df = []

for i, ref in enumerate(datasetRefs):
    if i < 10:
        print(f"========================datasetType = {data_product} ============================================")
        print("fullId.............................:",ref.dataId.full)
      
    visit = ref.dataId['visit']

    if FLAG_RESTRICT_VISITS:
        visitdate = visit//100_000
 
        if visitdate < VISITDATEMIN:
            continue
        if visitdate > VISITDATEMAX:
            continue

    if i%500 == 0:
        print(f"====================== {i} ==datasetType = {data_product} ============================================")
        print("fullId.............................:",ref.dataId.full)

    
    data = butler.get(ref)


    if isinstance(data,Table):
        t = data.asAstropy()
        df = t.to_pandas()
    else:
        df = data
    

    if(len(df)>0):
        df["visitId"] = visit
        all_df.append(df)
    

In [ ]:
#if FLAG_GET_SCHEMA:
#    print(data.getSchema())


In [ ]:
#data.getSchema()

In [ ]:
#df.asAstropy().to_pandas()

In [ ]:
df

In [ ]:
df = pd.concat(all_df)

In [ ]:
df

In [ ]:
df.to_csv(filesource_out)

In [ ]:
print(df.columns.tolist())

In [ ]:
df['visitId'].unique()